## Pacakge & Data Import

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm
import time
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd
import seaborn as sns
import os

In [2]:
# import labels for gudhi shapes
gudhi_shape_labels_unif = np.genfromtxt('Gudhi Shape Dataset Uniform/shape_labels.csv', delimiter=',', skip_header=1)
gudhi_shape_labels_unif = gudhi_shape_labels_unif.astype(int)[:,2]
print(len(gudhi_shape_labels_unif))

2000


In [3]:
# import labels for gudhi shapes
gudhi_shape_labels_nonunif = np.genfromtxt('Gudhi Shape Dataset Nonuniform/shape_labels.csv', delimiter=',', skip_header=1)
gudhi_shape_labels_nonunif = gudhi_shape_labels_nonunif.astype(int)[:,2]
print(len(gudhi_shape_labels_nonunif))

3157


/var/folders/d3/99lvl9fd1673ngz9966pvq100000gn/T/ipykernel_56357/2240478565.py:3: RuntimeWarning: invalid value encountered in cast
  gudhi_shape_labels_nonunif = gudhi_shape_labels_nonunif.astype(int)[:,2]


In [4]:
gudhi_shapes_nonunif = np.array([])

base = 'Gudhi Shape Dataset Nonuniform'

# Check if the required files exist for each shape
for i in range(1000, 4769):
    points_file = f'{base}/shape_{i}_points.csv'
    laplacian_file = f'{base}/shape_{i}_laplacian.csv'
    vr_persistence_image_file = f'{base}/shape_{i}_vr_persistence_image.csv'
    abstract_persistence_image_file = f'{base}/shape_{i}_abstract_persistence_image.csv'

    if (os.path.exists(points_file) and
        os.path.exists(laplacian_file) and
        os.path.exists(vr_persistence_image_file) and
        os.path.exists(abstract_persistence_image_file)):
        gudhi_shapes_nonunif = np.append(gudhi_shapes_nonunif, i)
    

gudhi_shapes_nonunif = gudhi_shapes_nonunif.astype(int)

In [5]:
# Uniform

seed_value = 221
np.random.seed(seed_value)

num_samples_unif = len(gudhi_shape_labels_unif) # currently set to full dataset
num_samples_unif = 1000

# Generate random indices
random_indices_unif = np.random.choice(len(gudhi_shape_labels_unif), size=num_samples_unif, replace=False)
base = 'Gudhi Shape Dataset Uniform'
# Select the corresponding data and labels
gudhi_laplacians_unif = []
gudhi_vr_persistence_images_unif = []
gudhi_abstract_persistence_images_unif = []
gudhi_selected_labels_unif = []

for i in random_indices_unif:
    gudhi_laplacians_unif.append(np.genfromtxt(f'{base}/shape_{i}_laplacian.csv', delimiter=',', skip_header=0))
    gudhi_vr_persistence_images_unif.append(np.genfromtxt(f'{base}/shape_{i}_vr_persistence_image.csv', delimiter=',', skip_header=0))
    gudhi_abstract_persistence_images_unif.append(np.genfromtxt(f'{base}/shape_{i}_abstract_persistence_image.csv', delimiter=',', skip_header=0))
    gudhi_selected_labels_unif.append(gudhi_shape_labels_unif[i])

# Convert selected labels to NumPy array
gudhi_selected_labels_unif = np.array(gudhi_selected_labels_unif)

# Print summary
print(f"Randomly selected {num_samples_unif} samples.")
print(f"Shape of laplacians: {np.array(gudhi_laplacians_unif).shape}")
print(f"Shape of VR persistence images: {np.array(gudhi_vr_persistence_images_unif).shape}")
print(f"Shape of abstract persistence images: {np.array(gudhi_abstract_persistence_images_unif).shape}")
print(f"Shape of selected labels: {gudhi_selected_labels_unif.shape}")

Randomly selected 1000 samples.
Shape of laplacians: (1000, 1000, 1000)
Shape of VR persistence images: (1000, 100, 100)
Shape of abstract persistence images: (1000, 100, 100)
Shape of selected labels: (1000,)


In [6]:
# Non-uniform

seed_value = 221
np.random.seed(seed_value)

num_samples_nonunif = len(gudhi_shapes_nonunif) # currently set to full dataset
num_samples_nonunif = 1000

# Generate random indices
random_indices_nonunif = np.random.choice(len(gudhi_shapes_nonunif), size=num_samples_nonunif, replace=False)
base = 'Gudhi Shape Dataset Nonuniform'
# Select the corresponding data and labels
gudhi_laplacians_nonunif = []
gudhi_vr_persistence_images_nonunif = []
gudhi_abstract_persistence_images_nonunif = []
gudhi_selected_labels_nonunif = []

for i in random_indices_nonunif:
    gudhi_laplacians_nonunif.append(np.genfromtxt(f'{base}/shape_{gudhi_shapes_nonunif[i]}_laplacian.csv', delimiter=',', skip_header=0))
    gudhi_vr_persistence_images_nonunif.append(np.genfromtxt(f'{base}/shape_{gudhi_shapes_nonunif[i]}_vr_persistence_image.csv', delimiter=',', skip_header=0))
    gudhi_abstract_persistence_images_nonunif.append(np.genfromtxt(f'{base}/shape_{gudhi_shapes_nonunif[i]}_abstract_persistence_image.csv', delimiter=',', skip_header=0))
    gudhi_selected_labels_nonunif.append(gudhi_shape_labels_nonunif[i])

# Convert selected labels to NumPy array
gudhi_selected_labels_nonunif = np.array(gudhi_selected_labels_nonunif)

# Print summary
print(f"Randomly selected {num_samples_nonunif} samples.")
print(f"Shape of laplacians: {np.array(gudhi_laplacians_nonunif).shape}")
print(f"Shape of VR persistence images: {np.array(gudhi_vr_persistence_images_nonunif).shape}")
print(f"Shape of abstract persistence images: {np.array(gudhi_abstract_persistence_images_nonunif).shape}")
print(f"Shape of selected labels: {gudhi_selected_labels_nonunif.shape}")

Randomly selected 1000 samples.
Shape of laplacians: (1000, 1000, 1000)
Shape of VR persistence images: (1000, 100, 100)
Shape of abstract persistence images: (1000, 100, 100)
Shape of selected labels: (1000,)


In [7]:
gudhi_laplacians = np.concatenate((gudhi_laplacians_nonunif, gudhi_laplacians_unif), axis=0)
gudhi_vr_persistence_images = np.concatenate((gudhi_vr_persistence_images_nonunif, gudhi_vr_persistence_images_unif), axis=0)
gudhi_abstract_persistence_images = np.concatenate((gudhi_abstract_persistence_images_nonunif, gudhi_abstract_persistence_images_unif), axis=0)
gudhi_selected_labels = np.concatenate((gudhi_selected_labels_nonunif, gudhi_selected_labels_unif), axis=0)

In [8]:
print(f"Shape of combined laplacians: {gudhi_laplacians.shape}")
print(f"Shape of combined VR persistence images: {gudhi_vr_persistence_images.shape}")
print(f"Shape of combined abstract persistence images: {gudhi_abstract_persistence_images.shape}")
print(f"Shape of combined selected labels: {gudhi_selected_labels.shape}")

Shape of combined laplacians: (2000, 1000, 1000)
Shape of combined VR persistence images: (2000, 100, 100)
Shape of combined abstract persistence images: (2000, 100, 100)
Shape of combined selected labels: (2000,)


In [9]:
# import labels for medical shapes
medical_shape_labels = np.genfromtxt('Medical Dataset/shape_labels.csv', delimiter=',', skip_header=1)
medical_shape_labels = medical_shape_labels.astype(int)[:,2]
print(len(medical_shape_labels))

162


In [10]:
seed_value = 221
np.random.seed(seed_value)

num_samples = 162 # currently set to full dataset

# Generate random indices
random_indices = np.random.choice(len(medical_shape_labels), size=num_samples, replace=False)
base = 'Medical Dataset/'
# Select the corresponding data and labels
medical_laplacians = []
medical_vr_persistence_images = []
medical_abstract_persistence_images = []
medical_selected_labels = []

for i in random_indices:
    medical_laplacians.append(np.genfromtxt(f'{base}/shape_{i}_laplacian.csv', delimiter=',', skip_header=0))
    medical_vr_persistence_images.append(np.genfromtxt(f'{base}/shape_{i}_vr_persistence_image.csv', delimiter=',', skip_header=0))
    medical_abstract_persistence_images.append(np.genfromtxt(f'{base}/shape_{i}_abstract_persistence_image.csv', delimiter=',', skip_header=0))
    medical_selected_labels.append(medical_shape_labels[i])

# Convert selected labels to NumPy array
medical_selected_labels = np.array(medical_selected_labels)

# Print summary
print(f"Randomly selected {num_samples} samples.")
print(f"Shape of laplacians: {np.array(medical_laplacians).shape}")
print(f"Shape of VR persistence images: {np.array(medical_vr_persistence_images).shape}")
print(f"Shape of abstract persistence images: {np.array(medical_abstract_persistence_images).shape}")
print(f"Shape of selected labels: {medical_selected_labels.shape}")

Randomly selected 162 samples.
Shape of laplacians: (162, 1000, 1000)
Shape of VR persistence images: (162, 100, 100)
Shape of abstract persistence images: (162, 100, 100)
Shape of selected labels: (162,)


## Define data class

In [11]:
class ShapeDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in data]
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


## Data Splitting

In [12]:
train_ratio = 0.8
valid_ratio = 0.1
test_ratio = 0.1

# Split Gudhi Laplacians
train_data_gudhi_laplacians, test_data_gudhi_laplacians, train_labels_gudhi, test_labels_gudhi = train_test_split(
    gudhi_laplacians, gudhi_selected_labels, test_size=(1 - train_ratio), random_state=42
)
valid_data_gudhi_laplacians, test_data_gudhi_laplacians, valid_labels_gudhi, test_labels_gudhi = train_test_split(
    test_data_gudhi_laplacians, test_labels_gudhi, test_size=(test_ratio / (valid_ratio + test_ratio)), random_state=42
)

# Split Medical Laplacians (valid/test split only, since all are used for validation/testing)
valid_data_medical_laplacians, test_data_medical_laplacians, valid_labels_medical, test_labels_medical = train_test_split(
    medical_laplacians, medical_selected_labels, test_size=0.5, random_state=42
)

# Combine validation sets for Laplacians and labels
valid_laplacians = valid_data_gudhi_laplacians
valid_labels = valid_labels_gudhi

# Combine test sets for completeness
test_laplacians = test_data_gudhi_laplacians
test_labels = test_labels_gudhi

# Print a summary
print(f"Laplacians Train data size: {len(train_data_gudhi_laplacians)}")
print(f"Laplacians Validation data size: {len(valid_laplacians)}")
print(f"Laplacians Test data size: {len(test_laplacians)}")


# Split VR Persistence Images
train_data_gudhi_vr_persistence_images, test_data_gudhi_vr_persistence_images, train_labels_gudhi_check, test_labels_gudhi_check = train_test_split(
    gudhi_vr_persistence_images, gudhi_selected_labels, test_size=(1 - train_ratio), random_state=42
)
valid_data_gudhi_vr_persistence_images, test_data_gudhi_vr_persistence_images, valid_labels_gudhi_check, test_labels_gudhi_check = train_test_split(
    test_data_gudhi_vr_persistence_images, test_labels_gudhi_check, test_size=(test_ratio / (valid_ratio + test_ratio)), random_state=42
)

# Split Medical VR Persistence Images (valid/test split only, since all are used for validation/testing)
valid_data_medical_vr_persistence_images, test_data_medical_vr_persistence_images, valid_labels_medical_check, test_labels_medical_check = train_test_split(
    medical_vr_persistence_images, medical_selected_labels, test_size=0.5, random_state=42
)

# Combine validation sets for VR Persistence Images and labels
valid_vr_persistence_images = valid_data_gudhi_vr_persistence_images
valid_vr_labels = valid_labels_gudhi

# Combine test sets for VR Persistence Images and labels
test_vr_persistence_images = test_data_gudhi_vr_persistence_images
test_vr_labels = test_labels_gudhi


# Split Abstract Persistence Images
train_data_gudhi_abstract_persistence_images, test_data_gudhi_abstract_persistence_images, train_labels_gudhi_check, test_labels_gudhi_check = train_test_split(
    gudhi_abstract_persistence_images, gudhi_selected_labels, test_size=(1 - train_ratio), random_state=42
)
valid_data_gudhi_abstract_persistence_images, test_data_gudhi_abstract_persistence_images, valid_labels_gudhi_check, test_labels_gudhi_check = train_test_split(
    test_data_gudhi_abstract_persistence_images, test_labels_gudhi_check, test_size=(test_ratio / (valid_ratio + test_ratio)), random_state=42
)

# Split Medical Abstract Persistence Images (valid/test split only, since all are used for validation/testing)
valid_data_medical_abstract_persistence_images, test_data_medical_abstract_persistence_images, valid_labels_medical_check, test_labels_medical_check = train_test_split(
    medical_abstract_persistence_images, medical_selected_labels, test_size=0.5, random_state=42
)

# Combine validation sets for Abstract Persistence Images and labels
valid_abstract_persistence_images = valid_data_gudhi_abstract_persistence_images
valid_abstract_labels = valid_labels_gudhi

# Combine test sets for Abstract Persistence Images and labels
test_abstract_persistence_images = test_data_gudhi_abstract_persistence_images
test_abstract_labels = test_labels_gudhi


Laplacians Train data size: 1600
Laplacians Validation data size: 200
Laplacians Test data size: 200


## CNN Definitions

In [13]:
class CNN(nn.Module):
    def __init__(self, input_shape, num_classes=2):
        super(CNN, self).__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        
        # Pooling Layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Adaptive Pooling to resize to 100x100
        self.adaptive_pool = nn.AdaptiveAvgPool2d((100, 100))
        
        # Dynamically calculate input size to fc1
        self.feature_size = self._get_feature_size(input_shape)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(self.feature_size, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def _get_feature_size(self, input_shape):
        # Create a dummy input to calculate size after conv and pooling
        dummy_input = torch.zeros(1, 1, *input_shape)
        x = self.pool(F.relu(self.conv1(dummy_input)))
        x = self.pool(F.relu(self.conv2(x)))
        
        # Apply adaptive pooling to get 100x100 size
        x = self.adaptive_pool(x)
        return x.numel()  # Number of elements after flattening

    def forward(self, x):
        # Apply convolutional layers with pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        # Apply adaptive pooling to resize to 100x100
        x = self.adaptive_pool(x)
        
        # Flatten and pass through fully connected layers
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [14]:
class DualInputCNN(nn.Module):
    def __init__(self, input_shape1, input_shape2, num_classes=2):
        super(DualInputCNN, self).__init__()

        # Laplacian input path with additional pooling to reduce to 100x100
        self.conv1_lap = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2_lap = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool_lap = nn.MaxPool2d(2, 2)  # Reduce spatial dimensions
        self.adaptive_pool_lap = nn.AdaptiveAvgPool2d((100, 100))  # Resize to 100x100
        
        # Persistence image input path (no pooling)
        self.conv1_pers = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2_pers = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.adaptive_pool_pers = nn.AdaptiveAvgPool2d((100, 100))  # Resize to 100x100
        
        # Fully connected layers
        self.fc1 = nn.Linear(32 * 100 * 100 + 32 * 100 * 100, 128)  # Adjusted for 100x100 input
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x1, x2):
        # Laplacians path (downsampling to 100x100)
        x1 = F.relu(self.conv1_lap(x1))
        x1 = self.pool_lap(x1)  # First pool: 250x250 -> 125x125
        x1 = F.relu(self.conv2_lap(x1))
        x1 = self.pool_lap(x1)  # Second pool: 125x125 -> 62x62
        x1 = self.adaptive_pool_lap(x1)  # Resize to 100x100
        
        # Persistence images path (no pooling)
        x2 = F.relu(self.conv1_pers(x2))
        x2 = F.relu(self.conv2_pers(x2))
        x2 = self.adaptive_pool_pers(x2)  # Ensure persistence images are 100x100
        
        # Concatenate along dim=1 (channels)
        x = torch.cat((x1, x2), dim=1)  # Concatenates the outputs along the channel axis

        # Flatten for fully connected layer
        x = torch.flatten(x, start_dim=1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

Training on cpu


## Data Preparation

In [16]:
# Convert data to PyTorch Datasets
train_dataset_laplacians = ShapeDataset(train_data_gudhi_laplacians, train_labels_gudhi)
valid_dataset_laplacians = ShapeDataset(valid_laplacians, valid_labels)
test_dataset_laplacians = ShapeDataset(test_laplacians, test_labels)

train_dataset_vr = ShapeDataset(train_data_gudhi_vr_persistence_images, train_labels_gudhi)
valid_dataset_vr = ShapeDataset(valid_vr_persistence_images, valid_vr_labels)
test_dataset_vr = ShapeDataset(test_vr_persistence_images, test_vr_labels)

train_dataset_abstract = ShapeDataset(train_data_gudhi_abstract_persistence_images, train_labels_gudhi)
valid_dataset_abstract = ShapeDataset(valid_abstract_persistence_images, valid_abstract_labels)
test_dataset_abstract = ShapeDataset(test_abstract_persistence_images, test_abstract_labels)

# Define DataLoaders
batch_size = 32

train_loader_laplacians = torch.utils.data.DataLoader(train_dataset_laplacians, batch_size=batch_size, shuffle=False)
valid_loader_laplacians = torch.utils.data.DataLoader(valid_dataset_laplacians, batch_size=batch_size, shuffle=False)
test_loader_laplacians = torch.utils.data.DataLoader(test_dataset_laplacians, batch_size=batch_size, shuffle=False)

train_loader_vr = torch.utils.data.DataLoader(train_dataset_vr, batch_size=batch_size, shuffle=False)
valid_loader_vr = torch.utils.data.DataLoader(valid_dataset_vr, batch_size=batch_size, shuffle=False)
test_loader_vr = torch.utils.data.DataLoader(test_dataset_vr, batch_size=batch_size, shuffle=False)

train_loader_abstract = torch.utils.data.DataLoader(train_dataset_abstract, batch_size=batch_size, shuffle=False)
valid_loader_abstract = torch.utils.data.DataLoader(valid_dataset_abstract, batch_size=batch_size, shuffle=False)
test_loader_abstract = torch.utils.data.DataLoader(test_dataset_abstract, batch_size=batch_size, shuffle=False)

## Model Instantiation

In [17]:
# For single-input CNN (Laplacians)
input_shape = train_data_gudhi_laplacians[0].shape
num_classes = 2 # binary classification

model_single_laplacians = CNN(input_shape, num_classes)

# For dual-input CNNs (Laplacians + VR Persistence Images, Laplacians + Abstract Persistence Images)
input_shape1 = train_data_gudhi_laplacians[0].shape
input_shape2 = train_data_gudhi_vr_persistence_images[0].shape
input_shape3 = train_data_gudhi_abstract_persistence_images[0].shape

model_dual_lap_vr = DualInputCNN(input_shape1, input_shape2, num_classes)
model_dual_lap_abstract = DualInputCNN(input_shape1, input_shape3, num_classes)

## Training, Validation, and Testing Functions

In [18]:
def train_single_input(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    progress_bar = tqdm(dataloader, desc="Training", leave=False)
    
    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        
        progress_bar.set_postfix(loss=loss.item())
    
    accuracy = correct / len(dataloader.dataset)
    return total_loss / len(dataloader.dataset), accuracy


def validate_single_input(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    progress_bar = tqdm(dataloader, desc="Validating", leave=False)
    
    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            
            progress_bar.set_postfix(loss=loss.item())
    
    accuracy = correct / len(dataloader.dataset)
    return total_loss / len(dataloader.dataset), accuracy


def train_dual_input(model, dataloader1, dataloader2, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    progress_bar = tqdm(zip(dataloader1, dataloader2), desc="Training (Dual Input)", leave=False, total=min(len(dataloader1), len(dataloader2)))
    
    for (inputs1, labels1), (inputs2, labels2) in progress_bar:
        inputs1, labels1 = inputs1.to(device), labels1.to(device)
        inputs2, labels2 = inputs2.to(device), labels2.to(device)
        
        if not torch.equal(labels1, labels2):
            print("Labels mismatch in dual-input training! Skipping batch.")
            continue
        
        optimizer.zero_grad()
        outputs = model(inputs1, inputs2)
        loss = criterion(outputs, labels1)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels1).sum().item()
        
        progress_bar.set_postfix(loss=loss.item())
    
    accuracy = correct / len(dataloader1.dataset)
    return total_loss / len(dataloader1.dataset), accuracy


def validate_dual_input(model, valid_loader_laplacians, valid_loader_vr, criterion, device='cuda'):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for (inputs1, _), (inputs2, targets) in zip(valid_loader_laplacians, valid_loader_vr):
            inputs1, inputs2, targets = inputs1.to(device), inputs2.to(device), targets.to(device)
            
            outputs = model(inputs1, inputs2)
            loss = criterion(outputs, targets)
            running_loss += loss.item()
            
            if outputs.shape[-1] > 1:
                _, predicted = torch.max(outputs, 1)
            else:  # Binary classification
                predicted = (outputs > 0.5).float()
            
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    avg_loss = running_loss / len(valid_loader_laplacians)
    accuracy = correct / total

    return avg_loss, accuracy

def test_single_input(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    progress_bar = tqdm(dataloader, desc="Testing", leave=False)

    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(dataloader.dataset)
    accuracy = correct / total
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    metrics_df = pd.DataFrame({
        'Loss': [avg_loss],
        'Accuracy': [accuracy],
        'Precision': [precision],
        'Recall': [recall],
        'F1 Score': [f1]
    })

    cm = confusion_matrix(all_labels, all_preds)

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
    return metrics_df, cm


def test_dual_input(model, dataloader1, dataloader2, criterion, device='cuda'):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    progress_bar = tqdm(zip(dataloader1, dataloader2), desc="Testing (Dual Input)", leave=False, total=min(len(dataloader1), len(dataloader2)))

    with torch.no_grad():
        for (inputs1, _), (inputs2, targets) in progress_bar:
            inputs1, inputs2, targets = inputs1.to(device), inputs2.to(device), targets.to(device)

            outputs = model(inputs1, inputs2)
            loss = criterion(outputs, targets)
            running_loss += loss.item()

            if outputs.shape[-1] > 1:
                _, predicted = torch.max(outputs, 1)
            else:  # Binary classification
                predicted = (outputs > 0.5).float()

            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    avg_loss = running_loss / len(dataloader1.dataset)
    accuracy = correct / total
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    metrics_df = pd.DataFrame({
        'Loss': [avg_loss],
        'Accuracy': [accuracy],
        'Precision': [precision],
        'Recall': [recall],
        'F1 Score': [f1]
    })

    cm = confusion_matrix(all_labels, all_preds)

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
    return metrics_df, cm

## Training Loops

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_single_laplacians.to(device)
model_dual_lap_vr.to(device)
model_dual_lap_abstract.to(device)

DualInputCNN(
  (conv1_lap): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_lap): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool_lap): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (adaptive_pool_lap): AdaptiveAvgPool2d(output_size=(100, 100))
  (conv1_pers): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_pers): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (adaptive_pool_pers): AdaptiveAvgPool2d(output_size=(100, 100))
  (fc1): Linear(in_features=640000, out_features=128, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer_single = optim.Adam(model_single_laplacians.parameters(), lr=0.001)
optimizer_dual_vr = optim.Adam(model_dual_lap_vr.parameters(), lr=0.001)
optimizer_dual_abstract = optim.Adam(model_dual_lap_abstract.parameters(), lr=0.001)

num_epochs = 10

In [21]:
# Initialize empty lists to store all metrics for each epoch
test_metrics_single_list = []
test_metrics_dual_vr_list = []
test_metrics_dual_abstract_list = []

# Initialize empty lists to store all confusion matrices for each epoch
cm_single_list = []
cm_dual_vr_list = []
cm_dual_abstract_list = []

# Training loop for single-input model (Laplacians)
for epoch in range(num_epochs):

      train_time_start = time.time()
      model_single_laplacians.train()  # Ensure the model is in training mode

      train_loss, train_acc = train_single_input(model_single_laplacians, train_loader_laplacians, optimizer_single, criterion, device)

      train_time_end = time.time()

      valid_time_start = time.time()

      model_single_laplacians.eval()  # Switch model to evaluation mode after training



      valid_loss, valid_acc = validate_single_input(model_single_laplacians, valid_loader_laplacians, criterion, device)

      valid_time_end = time.time()

      # Test the model after each epoch

      test_time_start = time.time()
      test_metrics_single, cm_single = test_single_input(model_single_laplacians, test_loader_laplacians, criterion, device)
      test_time_end = time.time()

      # Combine train, valid, and test metrics for this epoch
      metrics = test_metrics_single.copy()
      metrics['Train Loss'] = train_loss
      metrics['Train Accuracy'] = train_acc
      metrics['Valid Loss'] = valid_loss
      metrics['Valid Accuracy'] = valid_acc

      # Append to the list
      test_metrics_single_list.append(metrics)
      cm_single_list.append(cm_single)

      print(f"single-input model (Laplacians) Epoch {epoch+1}/{num_epochs}, "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
            f"Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

      torch.save(model_single_laplacians.state_dict(), "Trained CNNs/model_single_h0_lap_unif_nonunif_gudhi_epoch_{}.pth".format(epoch+1))

      print(f"single-input model (Laplacians)Training time: {train_time_end - train_time_start}")
      print(f"single-input model (Laplacians)Validation time: {valid_time_end - valid_time_start}")
      print(f"single-input model (Laplacians)Testing time: {test_time_end - test_time_start}")


# Training loop for dual-input model (Laplacians + VR Persistence Images)
for epoch in range(num_epochs):

      train_time_start = time.time()
      model_dual_lap_vr.train()  # Ensure the model is in training mode
      train_loss, train_acc = train_dual_input(model_dual_lap_vr, train_loader_laplacians, train_loader_vr, optimizer_dual_vr, criterion, device)
      train_time_end = time.time()
      valid_time_start = time.time()
      model_dual_lap_vr.eval()  # Switch model to evaluation mode after training


      valid_loss, valid_acc = validate_dual_input(model_dual_lap_vr, valid_loader_laplacians, valid_loader_vr, criterion, device)
      valid_time_end = time.time()

      # Test the model after each epoch
      test_time_start = time.time()
      test_metrics_dual_vr, cm_dual_vr = test_dual_input(model_dual_lap_vr, test_loader_laplacians, test_loader_vr, criterion, device)
      test_time_end = time.time()

      # Combine train, valid, and test metrics for this epoch
      metrics = test_metrics_dual_vr.copy()
      metrics['Train Loss'] = train_loss
      metrics['Train Accuracy'] = train_acc
      metrics['Valid Loss'] = valid_loss
      metrics['Valid Accuracy'] = valid_acc

      # Append to the list
      test_metrics_dual_vr_list.append(metrics)
      cm_dual_vr_list.append(cm_dual_vr)

      print(f"dual-input model (Laplacians + VR Persistence Images) Epoch {epoch+1}/{num_epochs}, "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
            f"Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

      torch.save(model_dual_lap_vr.state_dict(), "Trained CNNs/model_dual_h0_lap_vr_unif_nonunif_gudhi_epoch_{}.pth".format(epoch+1))

      print(f"dual-input model (Laplacians + VR Persistence Images) Training time: {train_time_end - train_time_start}")
      print(f"dual-input model (Laplacians + VR Persistence Images) Validation time: {valid_time_end - valid_time_start}")
      print(f"dual-input model (Laplacians + VR Persistence Images) Testing time: {test_time_end - test_time_start}")


# Training loop for dual-input model (Laplacians + Abstract Persistence Images)
for epoch in range(num_epochs):

      train_time_start = time.time()
      model_dual_lap_abstract.train()  # Ensure the model is in training mode
      train_loss, train_acc = train_dual_input(model_dual_lap_abstract, train_loader_laplacians, train_loader_abstract, optimizer_dual_abstract, criterion, device)
      train_time_end = time.time()

      valid_time_start = time.time()
      model_dual_lap_abstract.eval()  # Switch model to evaluation mode after training
      valid_loss, valid_acc = validate_dual_input(model_dual_lap_abstract, valid_loader_laplacians, valid_loader_abstract, criterion, device)
      valid_time_end = time.time()

      # Test the model after each epoch
      test_time_start = time.time()
      test_metrics_dual_abstract, cm_dual_abstract = test_dual_input(model_dual_lap_abstract, test_loader_laplacians, test_loader_abstract, criterion, device)
      test_time_end = time.time()

      # Combine train, valid, and test metrics for this epoch
      metrics = test_metrics_dual_abstract.copy()
      metrics['Train Loss'] = train_loss
      metrics['Train Accuracy'] = train_acc
      metrics['Valid Loss'] = valid_loss
      metrics['Valid Accuracy'] = valid_acc

      # Append to the list
      test_metrics_dual_abstract_list.append(metrics)
      cm_dual_abstract_list.append(cm_dual_abstract)

      print(f"dual-input model (Laplacians + Abstract Persistence Images) Epoch {epoch+1}/{num_epochs}, "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
            f"Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

      torch.save(model_dual_lap_abstract.state_dict(), "Trained CNNs/model_dual_h0_lap_abstract_unif_nonunif_gudhi_epoch_{}.pth".format(epoch+1))

      print(f"dual-input model (Laplacians + Abstract Persistence Images) Training time: {train_time_end - train_time_start}")
      print(f"dual-input model (Laplacians + Abstract Persistence Images) Validation time: {valid_time_end - valid_time_start}")
      print(f"dual-input model (Laplacians + Abstract Persistence Images) Testing time: {test_time_end - test_time_start}")


Training:   0%|          | 0/50 [00:00<?, ?it/s]

Validating:   0%|          | 0/7 [00:00<?, ?it/s]

Testing:   0%|          | 0/7 [00:00<?, ?it/s]

Test Loss: 0.0195, Test Accuracy: 0.9150, Precision: 0.8780, Recall: 0.9818, F1 Score: 0.9270
single-input model (Laplacians) Epoch 1/10, Train Loss: 0.1323, Train Acc: 0.7044, Valid Loss: 0.0190, Valid Acc: 0.8900
single-input model (Laplacians)Training time: 287.1641049385071
single-input model (Laplacians)Validation time: 7.847338914871216
single-input model (Laplacians)Testing time: 8.122156858444214


Training:   0%|          | 0/50 [00:00<?, ?it/s]

...

Testing (Dual Input):   0%|          | 0/7 [00:00<?, ?it/s]

Test Loss: 0.0090, Test Accuracy: 0.9450, Precision: 0.9231, Recall: 0.9818, F1 Score: 0.9515
dual-input model (Laplacians + Abstract Persistence Images) Epoch 9/10, Train Loss: 0.0044, Train Acc: 0.9544, Valid Loss: 0.1261, Valid Acc: 0.9350
dual-input model (Laplacians + Abstract Persistence Images) Training time: 301.1787431240082
dual-input model (Laplacians + Abstract Persistence Images) Validation time: 8.272717952728271
dual-input model (Laplacians + Abstract Persistence Images) Testing time: 8.785783052444458


Training (Dual Input):   0%|          | 0/50 [00:00<?, ?it/s]

Testing (Dual Input):   0%|          | 0/7 [00:00<?, ?it/s]

Test Loss: 0.0077, Test Accuracy: 0.9600, Precision: 0.9811, Recall: 0.9455, F1 Score: 0.9630
dual-input model (Laplacians + Abstract Persistence Images) Epoch 10/10, Train Loss: 0.0046, Train Acc: 0.9544, Valid Loss: 0.1123, Valid Acc: 0.9400
dual-input model (Laplacians + Abstract Persistence Images) Training time: 296.88099694252014
dual-input model (Laplacians + Abstract Persistence Images) Validation time: 8.111831903457642
dual-input model (Laplacians + Abstract Persistence Images) Testing time: 8.635580062866211


In [22]:
# Convert the lists of metrics into DataFrames with epoch as the index
test_metrics_single_df = pd.concat(test_metrics_single_list, ignore_index=True)
test_metrics_dual_vr_df = pd.concat(test_metrics_dual_vr_list, ignore_index=True)
test_metrics_dual_abstract_df = pd.concat(test_metrics_dual_abstract_list, ignore_index=True)

# Add 'Epoch' as new column
epochs = list(range(1, num_epochs + 1))

test_metrics_single_df['Epoch'] = epochs
test_metrics_dual_vr_df['Epoch'] = epochs
test_metrics_dual_abstract_df['Epoch'] = epochs

# Set 'Epoch' as index
test_metrics_single_df.set_index('Epoch', inplace=True)
test_metrics_dual_vr_df.set_index('Epoch', inplace=True)
test_metrics_dual_abstract_df.set_index('Epoch', inplace=True)

# Rename first two columns to 'Test Loss' and 'Test Accuracy'
test_metrics_single_df.rename(columns={'Loss': 'Test Loss', 'Accuracy': 'Test Accuracy'}, inplace=True)
test_metrics_dual_vr_df.rename(columns={'Loss': 'Test Loss', 'Accuracy': 'Test Accuracy'}, inplace=True)
test_metrics_dual_abstract_df.rename(columns={'Loss': 'Test Loss', 'Accuracy': 'Test Accuracy'}, inplace=True)

# Print final test metrics for all models
print("\nSingle-input model (Laplacians) Test Metrics:")
print(test_metrics_single_df)

print("\nDual-input model (Laplacians + VR Persistence Images) Test Metrics:")
print(test_metrics_dual_vr_df)

print("\nDual-input model (Laplacians + Abstract Persistence Images) Test Metrics:")
print(test_metrics_dual_abstract_df)



Single-input model (Laplacians) Test Metrics:
       Test Loss  Test Accuracy  Precision    Recall  F1 Score  Train Loss  \
Epoch                                                                        
1       0.019460          0.915   0.878049  0.981818  0.927039    0.132254   
2       0.017648          0.920   0.885246  0.981818  0.931034    0.008779   
3       0.009772          0.920   0.891667  0.972727  0.930435    0.008842   
4       0.009433          0.925   0.899160  0.972727  0.934498    0.007720   
5       0.009147          0.955   0.939130  0.981818  0.960000    0.006881   
6       0.008610          0.955   0.946903  0.972727  0.959641    0.007117   
7       0.008665          0.955   0.939130  0.981818  0.960000    0.006727   
8       0.009747          0.960   0.947368  0.981818  0.964286    0.006273   
9       0.009948          0.960   0.947368  0.981818  0.964286    0.006491   
10      0.010678          0.960   0.955357  0.972727  0.963964    0.006763   

       Train Acc

In [23]:
# Export test metrics to CSV files

test_metrics_single_df.to_csv('test_metrics_single_h0_lap_unif_nonunif_gudhi.csv')
test_metrics_dual_vr_df.to_csv('test_metrics_dual_h0_lap_vr_unif_nonunif_gudhi.csv')
test_metrics_dual_abstract_df.to_csv('test_metrics_dual_h0_lap_abstract_unif_nonunif_gudhi.csv')

In [ ]:
# Visualize and export confusion matrices to "Confusion\ Matrices/" folder

# Single-input model (Laplacians)
for i, cm in enumerate(cm_single_list):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={'size': 14})
    plt.title(f"Confusion Matrix - Single-input Model (Laplacians), Epoch {i+1}")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f"Confusion Matrices/single_h0_lap_unif_nonunif_gudhi_cm_epoch_{i+1}.png")
    plt.close()

# Dual-input model (Laplacians + VR Persistence Images)
for i, cm in enumerate(cm_dual_vr_list):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={'size': 14})
    plt.title(f"Confusion Matrix - Dual-input Model (Laplacians + VR Persistence Images), Epoch {i+1}")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f"Confusion Matrices/dual_h0_lap_vr_unif_nonunif_gudhi_cm_epoch_{i+1}.png")
    plt.close()

# Dual-input model (Laplacians + Abstract Persistence Images)
for i, cm in enumerate(cm_dual_abstract_list):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={'size': 14})
    plt.title(f"Confusion Matrix - Dual-input Model (Laplacians + Abstract Persistence Images), Epoch {i+1}")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f"Confusion Matrices/dual_h0_lap_abstract_unif_nonunif_gudhi_cm_epoch_{i+1}.png")
    plt.close()

: 